
Install Required Library

In [9]:
pip install openai==1.30


   ---------------------------------------- 0.0/320.7 kB ? eta -:--:--
   --- ------------------------------------ 30.7/320.7 kB 1.3 MB/s eta 0:00:01
   ------------------------- -------------- 204.8/320.7 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 320.7/320.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


Import Required Library


In [1]:
import azure.cognitiveservices.speech as speechsdk
import requests, uuid, json
import os
import openai

Speech to Text

In [23]:
AZURE_SPEECH_KEY='4914b71c272f4d1f895dcd115cd0eabe'
AZURE_SPEECH_REGION='eastus'
auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(languages=["en-US", "ta-IN", "zh-CN"])

speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY ,region=AZURE_SPEECH_REGION)
#speech_config.speech_recognition_language = 'ta-IN'
audio_config  = speechsdk.AudioConfig(use_default_microphone=True)

Speech_recongniser = speechsdk.SpeechRecognizer(speech_config=speech_config, auto_detect_source_language_config=auto_detect_source_language_config, audio_config=audio_config)

print('You can speak now')
speech_recognition_result = Speech_recongniser.recognize_once_async().get()
output = speech_recognition_result.text

You can speak now


Text Translation

In [24]:
key = "077cc1545b9a4fa8bec66cbe521f9e30"
endpoint = "https://api.cognitive.microsofttranslator.com/"
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'ta-IN',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': output
}]

request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json()

#print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
transOutput = response[0]['translations'][0]['text']

Send speech to Azure OpenAI

In [30]:

openai.api_type = "azure"
openai.api_base = "https://openai-rabogenai.openai.azure.com/"
openai.api_version = "2024-02-01"
openai.api_key = "caa4e583fc4047d58c62cc0de4739e10"

response = openai.completions.create(
  model='openai_deployment',
  prompt=transOutput,
  temperature=0,
  #max_tokens=10,
  #top_p=1,
  #frequency_penalty=0,
  #presence_penalty=0,
  #best_of=1,
  #stop=["\n"]
 )

openAiOutput = response.choices[0].text

Text to Audio

In [26]:
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config,audio_config=audio_config)
speech_synthesizer_result = speech_synthesizer.speak_text_async(openAiOutput).get()


In [ ]:
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

endpoint = "https://openai-rabogenai.openai.azure.com/"
deployment = 'openai_deployment'
      
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
      
client = AzureOpenAI(
    azure_endpoint=endpoint,
    #azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
)
      
completion = client.completions.create(
    model=deployment,
    prompt=transOutput
)
      
print(completion.to_json())

In [40]:
completion.choices[0].text

' Answer: Mahendra Singh Dhoni\n\nWho won the Man of Tournament in IPL'